In [43]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape

In [44]:
#set important variables (that don't change often)
start_date = '2004-01-01'
end_date   = '2015-12-31'

In [45]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
modis_product = 'MODIS/006/MOD11A2'
product = 'MOD11A2'
start_date = '2004-01-01'
end_date = '2004-12-31'
variables = ['LST_Day_1km'] #'QC_Day'

In [46]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [47]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [51]:
#for each product
for vari in variables:
    
    dat = imgs.filterDate(start_date, end_date).select(vari).toList(1000);
    
    #for each city
    for cit in range(1,2): #range(0,len(the_cities)):
        city_name = the_cities[cit][1]['properties']['Name']
        bbox = shape(the_cities[cit][1]['geometry']).bounds
        print vari, city_name, bbox
        
        city_name = city_name.replace(' ','')
        
        #create a polygon to clip things too
        # xMin, yMin, xMax, yMax
        thecity = ee.Geometry.Rectangle(bbox)
                  
        result = dat.get(0);
    
        #convert an image collection into an image
        for i in range(0,dat.length().getInfo()):
           result = ee.Image.cat([result, dat.get(i)])
        
        #result = result.clip(thecity)
        #request to download
        #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
        job_id = city_name + "_" + product +'_'+vari
        params = {
            'folder' : 'modis',
            'description' : job_id,
            'fileNamePrefix' : job_id,
            'region' : thecity
        }
        
        task = ee.batch.Export.image.toDrive(result, job_id, params )
        #task.start()   

LST_Day_1km dar es salaam (38.94993166800003, -7.127943276999929, 39.44200277300007, -6.552704129999938)


In [52]:
task.start()

In [23]:
help(ee.batch.Export.image.toDrive)

Help on function toDrive in module ee.batch:

toDrive(image, description='myExportImageTask', folder=None, fileNamePrefix=None, dimensions=None, region=None, scale=None, crs=None, crsTransform=None, maxPixels=None, shardSize=None, fileDimensions=None, skipEmptyTiles=None, **kwargs)
    Creates a task to export an EE Image to Drive.
    
    Args:
      image: The image to be exported.
      description: Human-readable name of the task.
      folder: The name of a unique folder in your Drive account to
          export into. Defaults to the root of the drive.
      fileNamePrefix: The Google Drive filename for the export.
          Defaults to the name of the task.
      dimensions: The dimensions of the exported image. Takes either a
          single positive integer as the maximum dimension or "WIDTHxHEIGHT"
          where WIDTH and HEIGHT are each positive integers.
      region: The lon,lat coordinates for a LinearRing or Polygon
          specifying the region to export. Can be sp